In [ ]:
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)[source]

# Question 1

## Question 1(a)

### Question 1(a)(i)

In [55]:
import csv
import pandas as pd
import pyomo.environ as pe
import numpy as np

In [56]:
pre0 = pd.read_csv('./ClassAssignment.csv', index_col = 0, names = range(7)) 
pre0_1 = pre0.drop(5, axis = 1, inplace = False)
pre0_2 = pre0_1.drop(6, axis = 1, inplace = False)
pre1 = pre0_2.drop('StudentNumber', axis = 0, inplace = False)
pre1

,1,2,3,4
0,,,,
1,2,1,3,4
2,1,2,4,3
3,3,4,1,2
4,2,3,1,4
5,3,1,4,2
...,...,...,...,...
96,4,1,3,2
97,4,2,1,3
98,4,3,2,1


In [57]:
students = range(1, 101)
classes = range(1, 5)

In [58]:
#set model
model1 = pe.ConcreteModel()
model1.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model1.x = pe.Var(
    model1.s, domain = pe.Binary)

#set objective
model1.obj = pe.Objective(
    expr = sum(model1.x[i, j] * int(pre1[j][i-1]) for i, j in model1.s),
    sense = pe.minimize)

In [59]:
# define constraints
def rule1(mod, i):
    return sum(mod.x[i, j] for j in classes) == 1
model1.const1 = pe.Constraint(students, rule = rule1)
def rule2(mod, j):
    return sum(mod.x[i, j] for i in students) >= 23
model1.const2 = pe.Constraint(classes, rule = rule2)
def rule3(mod, j):
    return sum(mod.x[i, j] for i in students) <= 27
model1.const3 = pe.Constraint(classes, rule = rule3)

### Question1(a)(ii)

In [60]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model1)

{'Problem': [{'Name': 'x401', 'Lower bound': 102.0, 'Upper bound': 102.0, 'Number of objectives': 1, 'Number of constraints': 109, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 1201, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.046863555908203125', 'Error rc': 0, 'Time': 0.38764047622680664}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [110]:
print('The value of optimal objective function is', model1.obj(),'\n')
print('The optimal solution is as follows:')
for i in students:
    for j in classes:
            if model1.x[i, j].value != 0:
                print('assign student', i, 'to class' ,j)

print('\nThe values of decision variables are as follows:')

for i in students:
    for j in classes:
        print(model1.x[i,j], model1.x[i, j].value)

The value of optimal objective function is 102.0 

The optimal solution is as follows:
assign student 1 to class 2
assign student 2 to class 1
assign student 3 to class 3
assign student 4 to class 3
assign student 5 to class 2
assign student 6 to class 3
assign student 7 to class 1
assign student 8 to class 2
assign student 9 to class 4
assign student 10 to class 3
assign student 11 to class 1
assign student 12 to class 1
assign student 13 to class 4
assign student 14 to class 4
assign student 15 to class 3
assign student 16 to class 1
assign student 17 to class 1
assign student 18 to class 1
assign student 19 to class 2
assign student 20 to class 2
assign student 21 to class 4
assign student 22 to class 4
assign student 23 to class 4
assign student 24 to class 2
assign student 25 to class 3
assign student 26 to class 1
assign student 27 to class 1
assign student 28 to class 4
assign student 29 to class 3
assign student 30 to class 4
assign student 31 to class 3
assign student 32 to cl

### Question1(a)(iii)

In [62]:
count1 = 0
count2 = 0
count3 = 0
count4 = 0
for i in students:
    for j in classes:
        if int(pre1[j][i-1]) == 1:
            if model1.x[i, j].value != 0:
                count1 += 1
        elif int(pre1[j][i-1]) == 2:
             if model1.x[i, j].value != 0:
                count2 += 1
        elif int(pre1[j][i-1]) == 3:
             if model1.x[i, j].value != 0:
                count3 += 1
        else:
             if model1.x[i, j].value != 0:
                count4 += 1
                
print(count1, 'student(s) will be assigned to their first choice class')
print(count2, 'student(s) will be assigned to their second choice class')
print(count3, 'student(s) will be assigned to their third choice class')
print(count4, 'student(s) will be assigned to their fourth choice class')

98 student(s) will be assigned to their first choice class
2 student(s) will be assigned to their second choice class
0 student(s) will be assigned to their third choice class
0 student(s) will be assigned to their fourth choice class


## Question 1(b)

### Question1(b)(i)

In [63]:
pre2 = pd.read_csv('./ClassAssignment.csv', index_col = 0)['Gender']
for i in students:
    if pre2[i] == 'F':
        pre2[i] = 0
    else:
        pre2[i] = 1
pre2

StudentNumber
1      0
2      0
3      0
4      0
5      0
      ..
96     1
97     1
98     1
99     1
100    1
Name: Gender, Length: 100, dtype: object

In [64]:
#set model
model2 = pe.ConcreteModel()
model2.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model2.x = pe.Var(
    model2.s, domain = pe.Binary)

#set objective
model2.obj = pe.Objective(
    expr = sum(model2.x[i, j] * int(pre1[j][i-1]) for i, j in model2.s),
    sense = pe.minimize)

In [65]:
# define constraints
model2.const1 = pe.Constraint(students, rule = rule1)
model2.const2 = pe.Constraint(classes, rule = rule2)
model2.const3 = pe.Constraint(classes, rule = rule3)

In [66]:
# add new constraints
def rule4(mod, j):
    return sum(model2.x[i, j] * pre2[i] for i in students) <= 4/9 * sum(model2.x[i, j] for i in students)
model2.const4 = pe.Constraint(classes, rule = rule4)
def rule5(mod, j):
    return sum(model2.x[i, j] * pre2[i] for i in students) >= 3/8 * sum(model2.x[i, j] for i in students)
model2.const5 = pe.Constraint(classes, rule = rule5)

### Question1(b)(ii)

In [68]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model2)

{'Problem': [{'Name': 'x401', 'Lower bound': 108.0, 'Upper bound': 108.0, 'Number of objectives': 1, 'Number of constraints': 117, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 2001, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.015636444091796875', 'Error rc': 0, 'Time': 0.14356136322021484}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [111]:
print('The value of optimal objective function is', model2.obj(),'\n')
print('The optimal solution is as follows:')
for i in students:
    for j in classes:
            if model2.x[i, j].value != 0:
                print('assign student', i, 'to class' ,j)

print('\nThe values of decision variables are as follows:')

for i in students:
    for j in classes:
        print(model2.x[i,j], model2.x[i, j].value)

The value of optimal objective function is 108.0 

The optimal solution is as follows:
assign student 1 to class 2
assign student 2 to class 1
assign student 3 to class 3
assign student 4 to class 3
assign student 5 to class 2
assign student 6 to class 3
assign student 7 to class 1
assign student 8 to class 2
assign student 9 to class 4
assign student 10 to class 3
assign student 11 to class 1
assign student 12 to class 1
assign student 13 to class 4
assign student 14 to class 4
assign student 15 to class 3
assign student 16 to class 1
assign student 17 to class 1
assign student 18 to class 1
assign student 19 to class 2
assign student 20 to class 2
assign student 21 to class 4
assign student 22 to class 4
assign student 23 to class 4
assign student 24 to class 2
assign student 25 to class 3
assign student 26 to class 1
assign student 27 to class 1
assign student 28 to class 4
assign student 29 to class 2
assign student 30 to class 4
assign student 31 to class 3
assign student 32 to cl

### Question1(b)(iii)

In [70]:
count2_1 = 0
count2_2 = 0
count2_3 = 0
count2_4 = 0
for i in students:
    for j in classes:
        if int(pre1[j][i-1]) == 1:
            if model2.x[i, j].value != 0:
                count2_1 += 1
        elif int(pre1[j][i-1]) == 2:
             if model2.x[i, j].value != 0:
                count2_2 += 1
        elif int(pre1[j][i-1]) == 3:
             if model2.x[i, j].value != 0:
                count2_3 += 1
        else:
             if model2.x[i, j].value != 0:
                count2_4 += 1
                
print(count2_1, 'student(s) will be assigned to their first choice class')
print(count2_2, 'student(s) will be assigned to their second choice class')
print(count2_3, 'student(s) will be assigned to their third choice class')
print(count2_4, 'student(s) will be assigned to their fourth choice class')

92 student(s) will be assigned to their first choice class
8 student(s) will be assigned to their second choice class
0 student(s) will be assigned to their third choice class
0 student(s) will be assigned to their fourth choice class


### Question1(b)(iv)

In [112]:
count2_1_1 = 0
count2_1_2 = 0
count2_2_1 = 0
count2_2_2 = 0
count2_3_1 = 0
count2_3_2 = 0
count2_4_1 = 0
count2_4_2 = 0

for i in students:
    for j in classes:
        if model2.x[i, j].value != 0:
            if j == 1:
                if pre2[i] == 0:
                    count2_1_1 += 1
                else:
                    count2_1_2 += 1
            elif j == 2:
                if pre2[i] == 0:
                    count2_2_1 += 1
                else:
                    count2_2_2 += 1
            elif j == 3:
                if pre2[i] == 0:
                    count2_3_1 += 1
                else:
                    count2_3_2 += 1
            else:
                if pre2[i] == 0:
                    count2_4_1 += 1
                else:
                    count2_4_2 += 1

print('there are', count2_1_1, 'boys and', count2_1_2, 'girls in class 1.')
print('there are', count2_2_1, 'boys and', count2_2_2, 'girls in class 2.')
print('there are', count2_3_1, 'boys and', count2_3_2, 'girls in class 3.')
print('there are', count2_4_1, 'boys and', count2_4_2, 'girls in class 4.')

there are 16 boys and 10 girls in class 1.
there are 14 boys and 11 girls in class 2.
there are 13 boys and 10 girls in class 3.
there are 16 boys and 10 girls in class 4.


## Question 1(c)

### Question1(c)(i)

In [72]:
#set model
model3 = pe.ConcreteModel()
model3.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model3.x = pe.Var(
    model3.s, domain = pe.Binary)

#set objective
model3.obj = pe.Objective(
    expr = sum(model3.x[i, j] * int(pre1[j][i-1]) for i, j in model3.s),
    sense = pe.minimize)

In [73]:
# define constraints
model3.const1 = pe.Constraint(students, rule = rule1)
model3.const2 = pe.Constraint(classes, rule = rule2)
model3.const3 = pe.Constraint(classes, rule = rule3)

In [74]:
# add new constraints
model3.const4 = pe.Constraint(
    expr = sum(model3.x[15, j] * model3.x[19, j] for j in classes) == 0)
model3.const5 = pe.Constraint(
    expr = sum(model3.x[60, j] * model3.x[68, j] for j in classes) == 0)

In [75]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model3)

{'Problem': [{'Name': 'x401', 'Lower bound': 102.0, 'Upper bound': 102.0, 'Number of objectives': 1, 'Number of constraints': 111, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 1201, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.015625', 'Error rc': 0, 'Time': 0.13437724113464355}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [113]:
print('The value of optimal objective function is', model3.obj(),'\n')

The value of optimal objective function is 102.0 



### Question1(c)(ii)

In [143]:
neighbor = [4, 8, 12, 25, 30, 36, 41, 48, 55, 59, 64, 69, 73, 89, 90, 96]

In [162]:
#set model
model4 = pe.ConcreteModel()
model4.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model4.x = pe.Var(
    model4.s, domain = pe.Binary)

#set objective
model4.obj = pe.Objective(
    expr = sum(model4.x[i, j] * int(pre1[j][i-1]) for i, j in model4.s),
    sense = pe.minimize)

In [163]:
# define constraints
model4.const1 = pe.Constraint(students, rule = rule1)
model4.const2 = pe.Constraint(classes, rule = rule2)
model4.const3 = pe.Constraint(classes, rule = rule3)

In [164]:
# add new constraints
def rule6(mod, j):
    return sum(model4.x[i,j] for i in neighbor) >= 3
model4.const4 = pe.Constraint(classes, rule = rule6)

In [165]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model4)

{'Problem': [{'Name': 'x401', 'Lower bound': 103.0, 'Upper bound': 103.0, 'Number of objectives': 1, 'Number of constraints': 113, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 1265, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0069484710693359375', 'Error rc': 0, 'Time': 0.12765717506408691}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [154]:
print('The value of optimal objective function is', model4.obj(),'\n')

The value of optimal objective function is 102.0 



In [166]:
print('The value of optimal objective function is', model4.obj(),'\n')
print('The optimal solution is as follows:')
for i in neighbor:
    for j in classes:
            if model4.x[i, j].value != 0:
                print('assign student', i, 'to class' ,j)

The value of optimal objective function is 103.0 

The optimal solution is as follows:
assign student 4 to class 3
assign student 8 to class 2
assign student 12 to class 1
assign student 25 to class 3
assign student 30 to class 4
assign student 36 to class 3
assign student 41 to class 1
assign student 48 to class 3
assign student 55 to class 1
assign student 59 to class 2
assign student 64 to class 2
assign student 69 to class 4
assign student 73 to class 2
assign student 89 to class 3
assign student 90 to class 2
assign student 96 to class 4


### Question1(c)(iii)

In [81]:
#set model
model5 = pe.ConcreteModel()
model5.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model5.x = pe.Var(
    model5.s, domain = pe.Binary)

#set objective
model5.obj = pe.Objective(
    expr = sum(model5.x[i, j] * int(pre1[j][i-1]) for i, j in model5.s),
    sense = pe.minimize)

In [82]:
# define constraints
model5.const1 = pe.Constraint(students, rule = rule1)
model5.const2 = pe.Constraint(classes, rule = rule2)
model5.const3 = pe.Constraint(classes, rule = rule3)

In [83]:
#add new constaints
model5.const4 = pe.Constraint(
    expr = sum(model5.x[1, j] * model5.x[4, j] for j in classes) == 1)
model5.const5 = pe.Constraint(
    expr = sum(model5.x[4, j] * model5.x[95, j] for j in classes) == 1)
model5.const6 = pe.Constraint(
    expr = model5.x[3, 1] == 1)
model5.const7 = pe.Constraint(
    expr = model5.x[40, 2] == 1)
model5.const8 = pe.Constraint(
    expr = model5.x[80, 3] == 1)
model5.const9 = pe.Constraint(
    expr = model5.x[99, 3] == 1)

In [84]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model5)

{'Problem': [{'Name': 'x401', 'Lower bound': 114.0, 'Upper bound': 114.0, 'Number of objectives': 1, 'Number of constraints': 115, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 1205, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.015710830688476562', 'Error rc': 0, 'Time': 0.14527463912963867}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [115]:
print('The value of optimal objective function is', model5.obj(),'\n')

The value of optimal objective function is 114.0 



## Question 1(d)

### Question1(d)(i)

In [85]:
friend = pd.read_csv('./ClassAssignment.csv') ['Friends']

In [86]:
friend2 = friend.copy()
friend3 = friend.copy()

for i in range(1,100):
    if friend[i] is not np.NaN:
        friend2[i] = friend[i].replace("[","").replace("]","").replace(" ","").split(",")  
        friend3[i] = []
        for j in friend2[i]:
            if j not in friend3[i]:
                friend3[i].append(j)

friend3

0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
              ...         
95      [4, 2, 32, 80, 78]
96    [27, 33, 11, 56, 83]
97     [55, 96, 53, 9, 92]
98    [29, 52, 55, 51, 27]
99    [83, 63, 67, 91, 13]
Name: Friends, Length: 100, dtype: object

In [87]:
#set model
model6 = pe.ConcreteModel()
model6.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model6.x = pe.Var(
    model6.s, domain = pe.Binary)

#set objective
model6.obj = pe.Objective(
    expr = sum(model6.x[i, j] * int(pre1[j][i-1]) for i, j in model6.s),
    sense = pe.minimize)

In [88]:
# define constraints
model6.const1 = pe.Constraint(students, rule = rule1)
model6.const2 = pe.Constraint(classes, rule = rule2)
model6.const3 = pe.Constraint(classes, rule = rule3)

In [89]:
students_f = range(9,100)

In [90]:
# add new constraints
def rule7(mod, i):
    return sum(model6.x[i+1,j] * model6.x[int(friend3[i][m]),j] for m in range(len(friend3[i])) for j in classes) >= 1
model6.const4 = pe.Constraint(students_f, rule = rule7)

def rule8(mod, i):
    return sum(model6.x[i+1,j] * model6.x[int(friend3[i][m]),j] for m in range(len(friend3[i])) for j in classes) <= 4
model6.const5 = pe.Constraint(students_f, rule = rule8)

### Question1(c)(ii)

In [91]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model6)

{'Problem': [{'Name': 'x401', 'Lower bound': 136.0, 'Upper bound': 136.0, 'Number of objectives': 1, 'Number of constraints': 291, 'Number of variables': 401, 'Number of binary variables': 400, 'Number of integer variables': 400, 'Number of continuous variables': 1, 'Number of nonzeros': 1201, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '1.0229759216308594', 'Error rc': 0, 'Time': 1.1758992671966553}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [117]:
print('The value of optimal objective function is', model6.obj(),'\n')
print('The optimal solution is as follows:')
for i in students:
    for j in classes:
            if model6.x[i, j].value >= 0.5:
                print('assign student', i, 'to class' ,j)

print('\nThe values of decision variables are as follows:')

for i in students:
    for j in classes:
        print(model6.x[i,j], model6.x[i, j].value)

The value of optimal objective function is 136.0 

The optimal solution is as follows:
assign student 1 to class 2
assign student 2 to class 4
assign student 3 to class 4
assign student 4 to class 3
assign student 5 to class 2
assign student 6 to class 3
assign student 7 to class 1
assign student 8 to class 2
assign student 9 to class 4
assign student 10 to class 1
assign student 11 to class 1
assign student 12 to class 4
assign student 13 to class 4
assign student 14 to class 2
assign student 15 to class 3
assign student 16 to class 1
assign student 17 to class 1
assign student 18 to class 1
assign student 19 to class 3
assign student 20 to class 2
assign student 21 to class 4
assign student 22 to class 1
assign student 23 to class 4
assign student 24 to class 4
assign student 25 to class 3
assign student 26 to class 4
assign student 27 to class 1
assign student 28 to class 4
assign student 29 to class 3
assign student 30 to class 1
assign student 31 to class 3
assign student 32 to cl

### Question1(c)(iii)

In [119]:
count4_1 = 0
count4_2 = 0
count4_3 = 0
count4_4 = 0
for i in students:
    for j in classes:
        if int(pre1[j][i-1]) == 1:
            if model6.x[i, j].value >= 0.5:
                count4_1 += 1
        elif int(pre1[j][i-1]) == 2:
             if model6.x[i, j].value >= 0.5:
                count4_2 += 1
        elif int(pre1[j][i-1]) == 3:
             if model6.x[i, j].value >= 0.5:
                count4_3 += 1
        else:
             if model6.x[i, j].value >= 0.5:
                count4_4 += 1
                
print(count4_1, 'student(s) will be assigned to their first choice class')
print(count4_2, 'student(s) will be assigned to their second choice class')
print(count4_3, 'student(s) will be assigned to their third choice class')
print(count4_4, 'student(s) will be assigned to their fourth choice class')

76 student(s) will be assigned to their first choice class
14 student(s) will be assigned to their second choice class
8 student(s) will be assigned to their third choice class
2 student(s) will be assigned to their fourth choice class


### Question1(c)(iv)

In [120]:
count = []
for i in students_f:
    countfriend = 0
    for j in classes:
        for m in range(len(friend3[i])):
            if model6.x[i, j].value * model6.x[int(friend3[i][m]), j].value == 1:
                countfriend += 1
    count.append(countfriend)

frequency = {}
for countfriend in count:
    if countfriend not in frequency:
        frequency[countfriend] = 1
    else:
        frequency[countfriend] += 1

print(frequency)
    

{0: 48, 1: 32, 2: 10, 3: 1}


In [121]:
print('There are', frequency[2] + frequency[3], 'students are assigned to the same class with more than one friends.')
print('There are', frequency[2], 'students are assigned to the same class with 2 friends.')
print('There are', frequency[3], 'students are assigned to the same class with 3 friends.')

There are 11 students are assigned to the same class with more than one friends.
There are 10 students are assigned to the same class with 2 friends.
There are 1 students are assigned to the same class with 3 friends.


## Question 1(e)

## Question 1(e)(i)

In [168]:
#set model
model7 = pe.ConcreteModel()
model7.s = pe.Set(initialize = list((i,j) for i in students for j in classes))

#set variable
model7.x = pe.Var(
    model7.s, domain = pe.Binary)

#set objective
model7.obj = pe.Objective(
    expr = sum(model7.x[i, j] * int(pre1[j][i-1]) for i, j in model7.s),
    sense = pe.minimize)

In [169]:
# define constraints
model7.const1 = pe.Constraint(students, rule = rule1)
model7.const2 = pe.Constraint(classes, rule = rule2)
model7.const3 = pe.Constraint(classes, rule = rule3)
model7.const4 = pe.Constraint(classes, rule = rule4)
model7.const5 = pe.Constraint(classes, rule = rule5)
model7.const6 = pe.Constraint(classes, rule = rule6)
model7.const7 = pe.Constraint(students_f, rule = rule7)
model7.const8 = pe.Constraint(students_f, rule = rule8)

In [170]:
model7.const9 = pe.Constraint(
    expr = sum(model7.x[15, j] * model7.x[19, j] for j in classes) == 0)
model7.const10 = pe.Constraint(
    expr = sum(model7.x[60, j] * model7.x[68, j] for j in classes) == 0)
model7.const11 = pe.Constraint(
    expr = sum(model7.x[1, j] * model7.x[4, j] for j in classes) == 1)
model7.const12 = pe.Constraint(
    expr = sum(model7.x[4, j] * model7.x[95, j] for j in classes) == 1)
model7.const13 = pe.Constraint(
    expr = model7.x[3, 1] == 1)
model7.const14 = pe.Constraint(
    expr = model7.x[40, 2] == 1)
model7.const15 = pe.Constraint(
    expr = model7.x[80, 3] == 1)
model7.const16 = pe.Constraint(
    expr = model7.x[99, 3] == 1)

In [171]:
#solve the model
solver = pe.SolverFactory('gurobi')
solver.solve(model7)

KeyError: 1465811842048

In [ ]:
print('The optimal solution is as follows:')
for i in students:
    for j in classes:
            if model7.x[i, j].value != 0:
                print('assign student', i, 'to class' ,j)
print('The value of optimal objective function is', model7.obj(),'\n')

## Question 1(e)(ii)

In [106]:
count5_1 = 0
count5_2 = 0
count5_3 = 0
count5_4 = 0
for i in students:
    for j in classes:
        if int(pre1[j][i-1]) == 1:
            if model7.x[i, j].value != 0:
                count5_1 += 1
        elif int(pre1[j][i-1]) == 2:
             if model7.x[i, j].value != 0:
                count5_2 += 1
        elif int(pre1[j][i-1]) == 3:
             if model7.x[i, j].value != 0:
                count5_3 += 1
        else:
             if model7.x[i, j].value != 0:
                count5_4 += 1
                
print(count5_1, 'student(s) will be assigned to their first choice class')
print(count5_2, 'student(s) will be assigned to their second choice class')
print(count5_3, 'student(s) will be assigned to their third choice class')
print(count5_4, 'student(s) will be assigned to their fourth choice class')

100 student(s) will be assigned to their first choice class
100 student(s) will be assigned to their second choice class
100 student(s) will be assigned to their third choice class
100 student(s) will be assigned to their fourth choice class
